In [4]:
data_pairs.head()

,simulant_id,household_id_census,first_name_census,middle_initial_census,last_name_census,street_number_census,street_name_census,unit_number_census,city_census,state_census,...,match,first_name_taxes,middle_initial_taxes,last_name_taxes,street_number_taxes,street_name_taxes,unit_number_taxes,city_taxes,state_taxes,zipcode_taxes
0,0_2,0_7,diana,p,kofron,5112,145th st,NaN,anytown,WA,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0_2,0_7,diana,p,kofron,5112,145th st,NaN,anytown,WA,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0_3,0_7,anna,a,kofron,5112,145th st,NaN,anytown,WA,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0_923,0_8033,gerald,r,butler,1130,mallory ln,NaN,anytown,WA,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0_2641,0_1066,loretta,t,carley,NaN,delacorte dr,NaN,anytown,WA,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
import pandas as pd
data_pairs = pd.read_csv('data_pairs.csv', low_memory=False)
data_pairs = data_pairs.drop(columns=["Unnamed: 0"])
data_pairs[data_pairs.select_dtypes(include=["object"]).columns] = (
    data_pairs.select_dtypes(include=["object"]).fillna("")
)

from textdistance import jaro_winkler, levenshtein

def calculate_similarity(row):
    row["first_name_similarity"] = jaro_winkler(row["first_name_census"], row["first_name_taxes"])
    row["last_name_similarity"] = jaro_winkler(row["last_name_census"], row["last_name_taxes"])
    row["street_number_similarity"] = levenshtein.normalized_similarity(row["street_number_census"], row["street_number_taxes"])
    row["street_name_similarity"] = levenshtein.normalized_similarity(row["street_name_census"], row["street_name_taxes"])
    row["city_similarity"] = levenshtein.normalized_similarity(row["city_census"], row["city_taxes"])
    row["zipcode_similarity"] = levenshtein.normalized_similarity(row["zipcode_census"], row["zipcode_taxes"])
    return row

# Apply similarity functions
data_pairs = data_pairs.apply(calculate_similarity, axis=1)

# Extract features and labels
X = data_pairs[["first_name_similarity", "last_name_similarity", "street_number_similarity", "street_name_similarity", "city_similarity", "zipcode_similarity"]].values
y = data_pairs["match"].values

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
model = models.Sequential([
    layers.Input(shape=(6,)),             # Six features
    layers.Dense(32, activation='relu'), # Hidden layer
    layers.Dropout(0.2),                 # Dropout to prevent overfitting
    layers.Dense(1, activation='sigmoid') # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X, y, epochs=10, batch_size=8, validation_split=0.2)




Epoch 1/10


1807/1807 [==============================] - 4s 2ms/step - loss: 0.1154 - accuracy: 0.9851 - val_loss: 0.0093 - val_accuracy: 0.9992
Epoch 2/10
1807/1807 [==============================] - 3s 2ms/step - loss: 0.0110 - accuracy: 0.9975 - val_loss: 0.0059 - val_accuracy: 0.9992
Epoch 3/10
1807/1807 [==============================] - 4s 2ms/step - loss: 0.0081 - accuracy: 0.9976 - val_loss: 0.0045 - val_accuracy: 0.9992
Epoch 4/10
1807/1807 [==============================] - 3s 2ms/step - loss: 0.0065 - accuracy: 0.9985 - val_loss: 0.0066 - val_accuracy: 0.9989
Epoch 5/10
1807/1807 [==============================] - 3s 2ms/step - loss: 0.0061 - accuracy: 0.9982 - val_loss: 0.0036 - val_accuracy: 0.9992
Epoch 6/10
1807/1807 [==============================] - 3s 2ms/step - loss: 0.0051 - accuracy: 0.9987 - val_loss: 0.0036 - val_accuracy: 0.9992
Epoch 7/10
1807/1807 [==============================] - 3s 2ms/step - loss: 0.0045 - accuracy: 0.9987 - val_loss: 0.0031 - val_accu

In [9]:
# Evaluate on the same dataset for simplicity
loss, accuracy = model.evaluate(X, y)
print(f"Loss: {loss}, Accuracy: {accuracy}")

565/565 [==============================] - 1s 1ms/step - loss: 0.0032 - accuracy: 0.9991
Loss: 0.0031933633144944906, Accuracy: 0.9990590810775757


In [ ]:
test_data = pd.DataFrame({
    "first_name_census": ["Jane", "John", "Jon", "Alice", "Robert"],
    "last_name_census": ["Doe", "Smith", "Smyth", "Smythe", "O'Neil"],
    "street_number_census": ["456", "123", "123", "123", "789"],
    "street_name_census": ["Oak St", "Elm St", "Elm Street", "Elm st.", "Pine St"],
    "city_census": ["Rockville", "Silver Spring", "Silver Spring", "Silvr Spring", "Bethesda"],
    "zipcode_census": ["11111", "22222", "22222", "22222", "33333"],

    "first_name_taxes": ["jane", "Jon", "John", "Alyce", "Robert"],
    "last_name_taxes": ["doe", "Smith", "Smith", "Smith", "Oneal"],
    "street_number_taxes": ["456", "123", "123", "123", "789"],
    "street_name_taxes": ["oak st", "elm st", "elm st", "ElmStreet", "Pine Street"],
    "city_taxes": ["rockville", "silver spring", "silver spring", "silver spring", "bethesda"],
    "zipcode_taxes": ["11111", "22222", "22222", "22222", "33333"],
})

string_cols = test_data.select_dtypes(include=["object", "string"]).columns
test_data[string_cols] = test_data[string_cols].apply(lambda col: col.str.lower())

# Apply similarity metrics
test_data = test_data.apply(calculate_similarity, axis=1)
X_test = test_data[["first_name_similarity", "last_name_similarity", "street_number_similarity", "street_name_similarity", "city_similarity", "zipcode_similarity"]].values

In [12]:
test_data.head()

,first_name_census,last_name_census,street_number_census,street_name_census,city_census,zipcode_census,first_name_taxes,last_name_taxes,street_number_taxes,street_name_taxes,city_taxes,zipcode_taxes,first_name_similarity,last_name_similarity,street_number_similarity,street_name_similarity,city_similarity,zipcode_similarity
0,jane,doe,456,oak st,rockville,11111,jane,doe,456,oak st,rockville,11111,1.0,1,1.0,1.0,1.0,1.0
1,john,smith,123,elm st,silver spring,22222,alice,smith,123,elm st,silver spring,22222,0.0,1,1.0,1.0,1.0,1.0


In [11]:
predictions = model.predict(X_test)
predicted_matches = (predictions > 0.5).astype(int)
print(predicted_matches)

1/1 [==============================] - 0s 91ms/step
[[0]
 [0]]


In [13]:
# Save the model
model.save("data_linkage_model.h5")

# Load the model
# loaded_model = tf.keras.models.load_model("data_linkage_model.h5")

C:\Users\dcjc9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
